In [147]:
import h5py

modelID = "record_7_model_nVKX8_"
weightsFileName = "../model_files/regressionJun24Positives/" + modelID + "modelWeights.h5"
weights = h5py.File(weightsFileName, 'r')
print weights.keys()

[u'activation_1', u'batchnormalization_1', u'convolution1d_1', u'dense_1', u'dense_2', u'dense_3', u'dropout_1', u'dropout_2', u'globalaveragepooling1d_1', u'prelu_1', u'prelu_2']


In [148]:
import numpy as np

layer = 'convolution1d_1'
# print np.array(weights[layer][str(layer + '_W')])
conv1_weights = np.array(weights[layer][str(layer + '_W')])
conv1_biases = np.array(weights[layer][str(layer + '_b')])
print conv1_weights.shape
print conv1_biases.shape

(19, 1, 4, 20)
(20,)


In [3]:
val_data_path = "../hdf5files/regression_jul3_strands_augmented_rep2only/valid_data.hdf5"
val_data = h5py.File(val_data_path)

In [4]:
print val_data['X']['sequence'].shape

(205984, 2000, 4)


In [125]:
import time
t0 = time.time()

numSequences = 6e6

filter_length = conv1_weights.shape[0]
stride = filter_length / 2
val_sequences = []
cnt = 0
for seq in val_data['X']['sequence']:
    if len(val_sequences) > numSequences:
        break
    seqLen = int(np.sum(seq))
    
    sliceIndices = np.arange(1000 - seqLen/2, 1000 + seqLen/2 - filter_length, stride)
    slicer = lambda t: [t, t + filter_length]
    sliceIndices = np.swapaxes(np.array(slicer(sliceIndices)), 0, 1)
    
    val_sequences += [seq[iin : iout] for iin, iout in sliceIndices]
    print_interval = int(3e5)
    if len(val_sequences) / print_interval > cnt / print_interval:
        cnt += len(sliceIndices)
        print "Took ~" + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt - print_interval) + " to " + str(cnt)
        t0 = time.time()
    else:
        cnt += len(sliceIndices)
#     if len(val_sequences) % 1000 == 0:
#             cnt = len(val_sequences)
#             print "Took " + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt-1000) + " to " + str(cnt)
            
#     for idx in startIndices:
#         val_sequences.append(seq[idx : idx + filter_length])
#         if len(val_sequences) % 1000 == 0:
#             cnt = len(val_sequences)
#             print "Took " + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt-1000) + " to " + str(cnt)

Took ~282.5163s for sequences 127 to 300127
Took ~269.5758s for sequences 300110 to 600110
Took ~260.6079s for sequences 600088 to 900088
Took ~263.1138s for sequences 900020 to 1200020
Took ~271.6573s for sequences 1200095 to 1500095
Took ~270.8824s for sequences 1500154 to 1800154
Took ~271.2648s for sequences 1800024 to 2100024
Took ~259.1446s for sequences 2100078 to 2400078
Took ~259.2118s for sequences 2400026 to 2700026
Took ~265.4643s for sequences 2700100 to 3000100
Took ~264.8328s for sequences 3000040 to 3300040
Took ~276.565s for sequences 3300168 to 3600168
Took ~281.2062s for sequences 3600018 to 3900018
Took ~279.9254s for sequences 3900114 to 4200114
Took ~274.4982s for sequences 4200102 to 4500102
Took ~280.4731s for sequences 4500084 to 4800084
Took ~273.964s for sequences 4800071 to 5100071
Took ~275.6245s for sequences 5100044 to 5400044
Took ~344.2208s for sequences 5400064 to 5700064
Took ~574.0349s for sequences 5700044 to 6000044


In [126]:
val_sequences = np.array(val_sequences)
print val_sequences.shape

(6000044, 19, 4)


In [150]:
import time
# from scipy.stats.mstats import zscore

numFilters = 20
conv1_biases_reshaped = np.broadcast_to(conv1_biases, shape = (19, 1, 4, numFilters))
conv1_weights_plus_biases = conv1_weights + conv1_biases_reshaped
# conv1_weights_plus_biases_normalized = zscore(conv1_weights_plus_biases, axis = None)

t0 = time.time()
filter_sequence_activations = np.tensordot(conv1_weights, val_sequences, axes = [[0, 2], [1, 2]])
# filter_sequence_activations = np.tensordot(conv1_weights_plus_biases, val_sequences, axes = [[0, 2], [1, 2]])
# filter_sequence_activations_normalized = np.tensordot(conv1_weights_plus_biases_normalized, val_sequences, axes = [[0, 2], [1, 2]])
print "Took " + str(round(time.time() - t0, 3)) + "s."

print filter_sequence_activations.shape

Took 1.159s.
(1, 20, 6000044)


In [151]:
# filter_sequence_activations[i, j] will be the activation of the ith filter on the jth sequence.

filter_sequence_activations = np.squeeze(filter_sequence_activations, axis=0)
print filter_sequence_activations.shape

(20, 6000044)


In [152]:
# normalize the filter activations so that we can use a unified activation threshold.
from scipy.stats.mstats import zscore
import scipy.stats as st

# the "globally" activating sequences are sequences that strongly activate their filters overall
# the "locally" activating sequences are the strongest sequences on a per-filter basis;
# this latter type allows to construct PWMs for all the filters, while the globally activating sequences
# only help with some PWMs

# global_activation_fraction = 0.0001
# global_activation_threshold = st.norm.ppf(1 - local_activation_fraction) # a z-score threshold for the entire array
# filter_sequence_activations_globally_normalized = zscore(filter_sequence_activations, axis=None)
# global_activating_sequences = np.ndarray(shape = filter_sequence_activations.shape, dtype = np.bool_)
# global_activating_sequences = filter_sequence_activations_globally_normalized > global_activation_threshold
# print global_activating_sequences.shape

local_activation_fraction = 0.0005 # get top 1 percent of sequences for each filter; approx 10K sequences
local_activation_threshold = st.norm.ppf(1 - local_activation_fraction)
filter_sequence_activations_locally_normalized = zscore(filter_sequence_activations, axis=1)
local_activating_sequences = np.ndarray(shape = filter_sequence_activations.shape, dtype = np.bool_)
local_activating_sequences = filter_sequence_activations_locally_normalized > local_activation_threshold
print local_activating_sequences.shape

(20, 6000044)


In [93]:
local_activating_sequences_val = st.norm.cdf(filter_sequence_activations_locally_normalized) > 0.99

In [153]:
print np.sum(global_activating_sequences, axis=1)
print np.sum(local_activating_sequences, axis=1)

[   572    695   9883    421   4226   2140   3073     42   6855      5
   3407  14019    489   3498   3322   5846   1051    500   2873   1878
   1397   2290  96548   2007   5073    301   1684   6122    996   2464
   1599    288   1314     42   1460    207   2728   4252     15   5072
    241     62    550   2490   3158    174     97   1965    170    593
    229   2485  23502     91     25    756   9006   2872 152145   1544
     83   2457   2299   4973     86   1197   1522   1545  11668    571
   3768  12892     30     35    108    539     34   3908  10155    119
   3930    208   1001      5   1765   5546   5524    101   1002   7931
   2883    981     52   6064   5545    345    266   1195      2  10657]
[5379 3982 6610 7289 2162 6552 2188 1496 4735 1414  869 2986  799 2434 2077
 2025 1157 1430 1979 4720]


In [154]:
filter_pwms = np.ndarray(shape = (19, 1, 4, numFilters))
for i in range(filter_pwms.shape[3]):
    filter_sequences = val_sequences[local_activating_sequences[i]]
    filter_pwm = np.sum(filter_sequences, axis = 0) / len(filter_sequences)
    filter_pwm = np.expand_dims(filter_pwm, axis = 1)
    filter_pwms[:, :, :, i] = filter_pwm
    
print filter_pwms.shape

(19, 1, 4, 20)


In [155]:
from scipy.stats import spearmanr
print spearmanr(np.ravel(filter_pwms), np.ravel(conv1_weights))

SpearmanrResult(correlation=0.88166671870545565, pvalue=0.0)


In [156]:
meme = open('../model_files/regression_jul3_strands_augmented/interpretation/nVKX8_20_conv_filters_meme_basset.txt', 'w')
# background frequencies 0.2901785   0.20715891  0.20895831  0.29370428
meme.write("MEME version 4\n\nALPHABET = ACGT\n\nStrands: + -\n\n" +
           "Background letter frequencies\nA 0.292 C 0.208 G 0.208 T 0.292\n\n")

filter_pwms_reaxis = np.squeeze(filter_pwms, axis = 1)
filter_pwms_reaxis = np.swapaxes(filter_pwms_reaxis, 0, 2)
filter_pwms_reaxis = np.swapaxes(filter_pwms_reaxis, 1, 2)
for i in range(len(filter_pwms_reaxis)):
    meme.write('MOTIF conv1_filter' + str(i) + '\n')
    meme.write('letter-probability matrix: alength= 4 w= 19\n')
    for j in range(len(filter_pwms_reaxis[i])):
        meme.write(' '.join(filter_pwms_reaxis[i, j].astype(np.str)) + '\n')
    meme.write('\n')

meme.close()

In [102]:
print val_sequences.shape
print local_activating_sequences.shape
print val_sequences[local_activating_sequences[:,:]].shape

(1000034, 19, 4)
(100, 1000034)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1000034 but corresponding boolean dimension is 100

In [77]:
print filter_sequence_activations[0][local_activating_sequences[0]]

[-4.13218212 -3.05381727 -3.68134642 ..., -3.92485142 -4.27456188
 -3.33619428]


# Misc

In [24]:
# test dot product speeds of numpy vs. theano

import numpy as np
import theano
import theano.tensor as T
import time

a = np.random.random((19,1,4,100))
b = np.random.random((1000000,19,4))

t0 = time.time()
nptest = np.tensordot(a, b, axes = [[0, 2], [1, 2]])
print nptest.shape
# print nptest
print "Took " + str(round(time.time() - t0, 3)) + "s."

t0 = time.time()
theanotest = T.tensordot(a, b, axes = [[0, 2], [1, 2]]).eval()
print theanotest.shape
# print theanotest
print "Took " + str(round(time.time() - t0, 3)) + "s."

(1, 100, 1000000)
Took 0.806s.
(1, 100, 1000000)
Took 17.453s.
